In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import os
import re
import json
import tensorflow as tf
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# Variables and data munging
train_positive_reviews = []
train_negative_reviews = []

test_positive_reviews = []
test_negative_reviews = []

for filename_ in os.listdir('../data/aclImdb/train/pos'):
    with open(os.path.join('../data/aclImdb/train/pos',filename_),'r') as f:
        sentiment = f.read()
        train_positive_reviews.append(sentiment)
    
for filename_ in os.listdir('../data/aclImdb/train/neg'):
    with open(os.path.join('../data/aclImdb/train/neg',filename_),'r') as f:
        sentiment = f.read()
        train_negative_reviews.append(sentiment)
    
for filename_ in os.listdir('../data/aclImdb/test/pos'):
    with open(os.path.join('../data/aclImdb/test/pos',filename_),'r') as f:
        sentiment = f.read()
        test_positive_reviews.append(sentiment)
    
for filename_ in os.listdir('../data/aclImdb/test/neg'):
    with open(os.path.join('../data/aclImdb/test/neg',filename_),'r') as f:
        sentiment = f.read()
        test_negative_reviews.append(sentiment)

train_reviews = train_positive_reviews + train_negative_reviews
test_reviews = test_positive_reviews + test_negative_reviews

train_labels = [0]*len(train_positive_reviews) + [1]*len(train_negative_reviews)
test_labels = [0]*len(test_positive_reviews) + [1]*len(test_negative_reviews)

print 'Number of training reviews: ', len(train_reviews)
print 'Number of testing reviews: ', len(test_reviews)
print 'Number of train labels: ', len(train_labels)
print 'Number of test labels: ', len(test_labels)

Number of training reviews:  25000
Number of testing reviews:  25000
Number of train labels:  25000
Number of test labels:  25000


In [3]:
# creating dataframes
reviews_train_df = pd.DataFrame({'reviews': train_reviews, 'sentiment':train_labels})
reviews_test_df = pd.DataFrame({'reviews': test_reviews, 'sentiment':test_labels})

reviews_train_df = reviews_train_df.sample(frac=1).reset_index(drop=True)
reviews_test_df = reviews_test_df.sample(frac=1).reset_index(drop=True)

# 1 for negative and 0 for positive
print reviews_train_df.head()
print reviews_test_df.head()

                                             reviews  sentiment
0  What are people on here talking about? I must ...          1
1  This is a fine drama and a nice change of pace...          0
2  A thematic staple of cinema since its inceptio...          1
3  I read somewhere where this film was supposed ...          1
4  I am Black American and I loved this soap oper...          0
                                             reviews  sentiment
0  Beats me how people can describe this adolesce...          1
1  Hitchcock is a great director. Ironically I mo...          1
2  A trite fish-out-of-water story about two frie...          1
3  I saw this superb documentary at the Santa Bar...          0
4  Well, 1st off I haven't seen "Silence of the L...          1


In [4]:
reviews = train_reviews + test_reviews
labels = train_labels + test_labels

MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# one hot encoding
labels = np_utils.to_categorical(np.asarray(labels))

print data.shape
print labels.shape

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:25000]
y_train = labels[:25000]
x_val = data[25000:]
y_val = labels[25000:]

/usr/local/lib/python2.7/site-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 124259 unique tokens.
(50000, 500)
(50000, 2)


In [5]:
embeddings_index = {}
f = open('../data/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.array(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM), dtype='float32')
for word,i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [6]:
embedding_matrix.dtype

dtype('float32')

In [7]:
y_train.dtype

dtype('float64')

In [31]:
import tensorflow as tf

batchSize = 128
numClasses = 2
training_iters = 25000
learning_rate = 0.01
display_step = 5
tf.reset_default_graph()

In [32]:
input_data = tf.placeholder(tf.int32, [batchSize, MAX_SEQUENCE_LENGTH], name="input_x")
labels = tf.placeholder(tf.float32, [batchSize, numClasses], name="input_y")
data = tf.Variable(tf.zeros([batchSize, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM]), dtype=tf.float32)
data = tf.nn.embedding_lookup(embedding_matrix, input_data)

conv1 = tf.layers.conv1d(data,filters=32,kernel_size=(3),padding="same",activation=tf.nn.relu)
pool1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=(2), strides=2)
pool1_flat = tf.reshape(pool1, [-1, 8000])
dense1 = tf.layers.dense(inputs=pool1_flat, units=250, activation=tf.nn.relu)
out = tf.layers.dense(inputs=dense1, units=2, activation=tf.nn.softmax)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(out,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while (step * batchSize) < training_iters-1:
        batch_xs, batch_ys = x_train[(step-1)*batchSize: step*batchSize],y_train[(step-1)*batchSize: step*batchSize]
        # Fit training using batch data
        sess.run(optimizer, feed_dict={input_data: batch_xs, labels: batch_ys})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={input_data: batch_xs, labels: batch_ys})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={input_data: batch_xs, labels: batch_ys})
            print "Iter " + str(step*batchSize) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

Iter 640, Minibatch Loss= 0.797637, Training Accuracy= 0.51562
Iter 1280, Minibatch Loss= 0.844512, Training Accuracy= 0.46875
Iter 1920, Minibatch Loss= 0.828887, Training Accuracy= 0.48438
Iter 2560, Minibatch Loss= 0.742949, Training Accuracy= 0.57031
Iter 3200, Minibatch Loss= 0.813262, Training Accuracy= 0.50000
Iter 3840, Minibatch Loss= 0.852324, Training Accuracy= 0.46094
Iter 4480, Minibatch Loss= 0.766387, Training Accuracy= 0.54688
Iter 5120, Minibatch Loss= 0.813262, Training Accuracy= 0.50000
Iter 5760, Minibatch Loss= 0.789824, Training Accuracy= 0.52344
Iter 6400, Minibatch Loss= 0.875761, Training Accuracy= 0.43750
Iter 7040, Minibatch Loss= 0.805449, Training Accuracy= 0.50781
Iter 7680, Minibatch Loss= 0.828887, Training Accuracy= 0.48438
Iter 8320, Minibatch Loss= 0.789824, Training Accuracy= 0.52344
Iter 8960, Minibatch Loss= 0.828887, Training Accuracy= 0.48438
Iter 9600, Minibatch Loss= 0.844511, Training Accuracy= 0.46875
Iter 10240, Minibatch Loss= 0.891386, Tra